In [151]:
# creating paths to src and data folders in the repo
import sys
import pathlib
import os
import shutil
src_path = pathlib.Path().absolute().parent.parent / "src"
data_path = pathlib.Path().absolute().parent.parent / "data"

# add src path to sys.path so it is searched in import statements
sys.path.append(str(src_path))

# basic imports for data retreival and manipulation
import requests
import pandas as pd
import numpy as np
import datetime as dt
import time
import re

# importing api wrappers for reddit data
import praw
from psaw import PushshiftAPI

# retrieve api credentials from .gitignore'd text file
secrets_path = src_path / 'api_credentials.txt'
secrets_txt = open(secrets_path, 'r')

my_id = secrets_txt.readline().split('=')[1].rstrip()
my_secret = secrets_txt.readline().split('=')[1].rstrip()

secrets_txt.close()

# create a praw and pushshitft instances
reddit = praw.Reddit(
     client_id=my_id,
     client_secret=my_secret,
     user_agent="test_script by u/Mizule_RL"
 )

s_api = PushshiftAPI(reddit)


I'm choosing to limit my classes to: Digital, Paint, Ink, Pencil, Charcoal, Sculpture

In [202]:
def get_wordlists():
    '''
    returns a dictionary containing regex patterns of the keywords for each class
    '''
    wordlists = {'digital': ['digital', 'adobe', 'photoshop', 'procreate', 'wacom', 'tablet', 'illustrator', '3d', 'vector'],
                 'paint': ['acrylic', 'oil', 'watercolor', 'water color', 'watercolour', 'water colour'],
                 'non_ink_drawing': ['pencil', 'colored pencil', 'coloredpencil', 'color pencil', 'colorpencil', 'colour pencil', 'colourpencil', 'coloured pencil', 'colouredpencil', 'graphite', 'charcoal', 'chalk'],
                 'ink': ['pen(?!cil)', 'marker', 'ink'],
                 'sculpture': ['clay', 'string', 'sculpture', 'wire', 'nail', 'glass', 'yarn', 'metal', 'copper'],
                 'mixed_medium': []}
    return wordlists

In [203]:
def validate_submission(post):
    '''
    function for determining if a post is formated in a way that can be parsed properly
    
    Parameters:
    -- post: praw.submission object to be validated
    
    Returns:
    -- Boolean: True if post has valid formatting, False if not
    '''
    if post.selftext in ['[deleted]', '[removed]']:
        return False
    link_format = 'https://i\.redd\.it/.{13}\.(jpg|png)'
    corpus = []
    wordlists = get_wordlists()
    for value in wordlists.values():
        corpus.extend(value)
    title = post.title.lower().split(',')
    if len(title) == 4:
        medium = title[2].strip()
        if medium == 'me':
            medium = title[1].strip()
        if re.match(link_format, post.url):
            for word in corpus:
                if medium.find(word) >= 0:
                    return True
    return False
    
def extract_medium_from_title(title):
    '''
    identifies the medium given the title of the post
    '''
    medium_counter = 0
    medium_type = 'other'
    title_split = title.lower().split(',')
    medium = title_split[2].strip()
    if medium == 'me':
        medium = title_split[1].strip()
    wordlists = get_wordlists()
    for item in wordlists.items():
        for word in item[1]:
            if re.search(word, medium) != None:
                medium_type = item[0]
                medium_counter += 1
                break
    if medium_counter > 1:
        return 'mixed_medium'
    else:
        return medium_type

In [188]:
def make_post_dict(post):
    '''
    creates and returns a dict containing relevant data from a post
    '''
    post_dict = {'title': post.title,
                 'medium': extract_medium_from_title(post.title),
                 'url': post.url, 
                 'id': post.id,
                 'unix_time': int(post.created_utc),
                 'post': post}
    return post_dict

In [29]:
def fetch_submissions(min_posts, date = int(dt.datetime.now().timestamp())):
    start_epoch = date
    data_size = min_posts

    collected_posts = []
    while len(collected_posts) < data_size:
        print(f'polling pushshift for {data_size - len(collected_posts)} more posts before {start_epoch}')
        batch = list(s_api.search_submissions(before=start_epoch, subreddit='Art', limit=1000))
        for post in batch:
            if validate_submission(post):
                collected_posts.append(post)
        start_epoch = int(batch[-1].created_utc)
    return collected_posts

In [30]:
submissions_raw = fetch_submissions(10000)

polling pushshift for 10000 more posts before 1605647143
polling pushshift for 9959 more posts before 1605599241
polling pushshift for 9917 more posts before 1605585967
polling pushshift for 9862 more posts before 1605574629
polling pushshift for 9822 more posts before 1605565187
polling pushshift for 9783 more posts before 1605559306
polling pushshift for 9739 more posts before 1605551360
polling pushshift for 9694 more posts before 1605543522
polling pushshift for 9651 more posts before 1605537366
polling pushshift for 9602 more posts before 1605527150
polling pushshift for 9556 more posts before 1605512188
polling pushshift for 9512 more posts before 1605499991
polling pushshift for 9468 more posts before 1605488196
polling pushshift for 9413 more posts before 1605478748
polling pushshift for 9357 more posts before 1605470480
polling pushshift for 9305 more posts before 1605463098
polling pushshift for 9256 more posts before 1605455457
polling pushshift for 9197 more posts before 16

polling pushshift for 3074 more posts before 1603929914
polling pushshift for 3017 more posts before 1603922634
polling pushshift for 2971 more posts before 1603917654
polling pushshift for 2917 more posts before 1603910415
polling pushshift for 2868 more posts before 1603902956
polling pushshift for 2827 more posts before 1603895308
polling pushshift for 2779 more posts before 1603881442
polling pushshift for 2733 more posts before 1603859298
polling pushshift for 2687 more posts before 1603847110
polling pushshift for 2638 more posts before 1603837388
polling pushshift for 2594 more posts before 1603828840
polling pushshift for 2548 more posts before 1603822944
polling pushshift for 2499 more posts before 1603813353
polling pushshift for 2448 more posts before 1603804778
polling pushshift for 2394 more posts before 1603791495
polling pushshift for 2353 more posts before 1603774306
polling pushshift for 2310 more posts before 1603763542
polling pushshift for 2266 more posts before 160

In [39]:
submissions_dict = [make_post_dict(post) for post in submissions_raw]

In [40]:
submissions_dict

[{'title': 'Diya, me, 3D, 2020',
  'medium': 'digital',
  'url': 'https://i.redd.it/dk2s8oj6hsz51.jpg',
  'id': 'jvrq4w',
  'unix_time': 1605614029},
 {'title': 'Hexes&Exes, Webtoon, Digital Art, 2020',
  'medium': 'digital',
  'url': 'https://i.redd.it/o3m1hkvnesz51.png',
  'id': 'jvrk3v',
  'unix_time': 1605613143},
 {'title': 'Onward, me, ink on bristol, 2020',
  'medium': 'ink',
  'url': 'https://i.redd.it/0c4926h8dsz51.jpg',
  'id': 'jvrgof',
  'unix_time': 1605612655},
 {'title': 'Explosion, Me, Digital, 2020',
  'medium': 'digital',
  'url': 'https://i.redd.it/3txx4lcqbsz51.jpg',
  'id': 'jvrd6y',
  'unix_time': 1605612143},
 {'title': 'Lily, Me, Pen, 2020',
  'medium': 'ink',
  'url': 'https://i.redd.it/jxhc3pnmbsz51.jpg',
  'id': 'jvrczf',
  'unix_time': 1605612111},
 {'title': 'Imposter syndrome, me, digital, 2020',
  'medium': 'digital',
  'url': 'https://i.redd.it/buyuzygebsz51.jpg',
  'id': 'jvrcgp',
  'unix_time': 1605612035},
 {'title': 'White Rhino Track, Me, acrylic in

In [41]:
submissions_df = pd.DataFrame(submissions_dict)

In [42]:
submissions_df

,title,medium,url,id,unix_time
0,"Diya, me, 3D, 2020",digital,https://i.redd.it/dk2s8oj6hsz51.jpg,jvrq4w,1605614029
1,"Hexes&Exes, Webtoon, Digital Art, 2020",digital,https://i.redd.it/o3m1hkvnesz51.png,jvrk3v,1605613143
2,"Onward, me, ink on bristol, 2020",ink,https://i.redd.it/0c4926h8dsz51.jpg,jvrgof,1605612655
3,"Explosion, Me, Digital, 2020",digital,https://i.redd.it/3txx4lcqbsz51.jpg,jvrd6y,1605612143
4,"Lily, Me, Pen, 2020",ink,https://i.redd.it/jxhc3pnmbsz51.jpg,jvrczf,1605612111
...,...,...,...,...,...
10031,"Untitled, me, acrylic on canvas board, 2020",paint,https://i.redd.it/hkzkihqtyau51.jpg,jewxsy,1603222750
10032,"Wire wrapped crystal pendant, me, argentium si...",sculpture,https://i.redd.it/fp8aqleqyau51.jpg,jewxdr,1603222716
10033,"“Lost”, Artist me, Digital Painting, 2020",digital,https://i.redd.it/5zrygut4yau51.jpg,jewv1b,1603222527
10034,"The great egret, me, watercolor, 2020",paint,https://i.redd.it/3pw64tibxau51.jpg,jewro8,1603222239


In [43]:
submissions_df.medium.value_counts()

digital         5094
paint           2484
mixed_medium    1145
ink              949
pencil           158
charcoal         104
sculpture        102
Name: medium, dtype: int64

In [48]:
submissions_df[submissions_df.id == 'jjlfqf']

,title,medium,url,id,unix_time
7219,"Gluttony, Nazeroo(me), Digital Art, 2020",digital,https://i.redd.it/o5c323tydtv51.jpg,jjlfqf,1603881778


In [50]:
submissions_dict[7220]

{'title': 'Self-portrait, Me, Digital, 2020',
 'medium': 'digital',
 'url': 'https://i.redd.it/jrhr0vpbdtv51.png',
 'id': 'jjldco',
 'unix_time': 1603881445}

In [51]:
remaining_posts = submissions_dict[7220:]

download all the images from the verified posts

In [52]:
count = 0
for post in remaining_posts:
    response = requests.get(post['url'])
    file_name = f"{post['id']}.{post['url'][-3:]}"
    file_path = data_path / 'initial_dump' / post['medium'] / file_name
    print(f'writting file: {file_name} | {len(remaining_posts) - count} images remaining')
    file = open(str(file_path), "wb")
    file.write(response.content)
    file.close()
    count += 1

writting file: jjldco.png | 2816 images remaining
writting file: jjkx8c.jpg | 2815 images remaining
writting file: jjkwkv.jpg | 2814 images remaining
writting file: jjkw2i.jpg | 2813 images remaining
writting file: jjks9m.jpg | 2812 images remaining
writting file: jjkqe4.jpg | 2811 images remaining
writting file: jjkoza.jpg | 2810 images remaining
writting file: jjklkw.jpg | 2809 images remaining
writting file: jjkiwi.jpg | 2808 images remaining
writting file: jjkir3.jpg | 2807 images remaining
writting file: jjkgao.jpg | 2806 images remaining
writting file: jjkews.jpg | 2805 images remaining
writting file: jjkbx5.jpg | 2804 images remaining
writting file: jjk6xj.jpg | 2803 images remaining
writting file: jjk4yl.jpg | 2802 images remaining
writting file: jjk48n.jpg | 2801 images remaining
writting file: jjk1np.jpg | 2800 images remaining
writting file: jjjy8r.jpg | 2799 images remaining
writting file: jjjqc2.jpg | 2798 images remaining
writting file: jjjo1j.jpg | 2797 images remaining


writting file: jj9rdn.jpg | 2652 images remaining
writting file: jj9otn.jpg | 2651 images remaining
writting file: jj9n7j.png | 2650 images remaining
writting file: jj9luh.jpg | 2649 images remaining
writting file: jj9js0.jpg | 2648 images remaining
writting file: jj9eet.png | 2647 images remaining
writting file: jj9cka.png | 2646 images remaining
writting file: jj9c8l.jpg | 2645 images remaining
writting file: jj99mf.jpg | 2644 images remaining
writting file: jj9889.png | 2643 images remaining
writting file: jj97uv.jpg | 2642 images remaining
writting file: jj97ak.jpg | 2641 images remaining
writting file: jj94xw.jpg | 2640 images remaining
writting file: jj91vo.jpg | 2639 images remaining
writting file: jj90oe.png | 2638 images remaining
writting file: jj90ms.jpg | 2637 images remaining
writting file: jj8xl7.jpg | 2636 images remaining
writting file: jj8vaa.jpg | 2635 images remaining
writting file: jj8rx0.jpg | 2634 images remaining
writting file: jj8o6j.jpg | 2633 images remaining


writting file: jj159e.jpg | 2487 images remaining
writting file: jj13b9.jpg | 2486 images remaining
writting file: jj0ypf.png | 2485 images remaining
writting file: jj0yon.jpg | 2484 images remaining
writting file: jj0s2o.png | 2483 images remaining
writting file: jj0rco.png | 2482 images remaining
writting file: jj0qr1.jpg | 2481 images remaining
writting file: jj0qjx.png | 2480 images remaining
writting file: jj0ms0.jpg | 2479 images remaining
writting file: jj0k8r.jpg | 2478 images remaining
writting file: jj0gdl.png | 2477 images remaining
writting file: jj0bas.jpg | 2476 images remaining
writting file: jj09fo.jpg | 2475 images remaining
writting file: jj093z.jpg | 2474 images remaining
writting file: jj05cc.jpg | 2473 images remaining
writting file: jj03zd.jpg | 2472 images remaining
writting file: jj02to.jpg | 2471 images remaining
writting file: jj01h2.jpg | 2470 images remaining
writting file: jizzke.jpg | 2469 images remaining
writting file: jizz93.jpg | 2468 images remaining


writting file: jir7vt.jpg | 2323 images remaining
writting file: jir77k.jpg | 2322 images remaining
writting file: jir4p0.jpg | 2321 images remaining
writting file: jir2nr.jpg | 2320 images remaining
writting file: jir10g.jpg | 2319 images remaining
writting file: jir0qq.jpg | 2318 images remaining
writting file: jiqy63.jpg | 2317 images remaining
writting file: jiqxq1.jpg | 2316 images remaining
writting file: jiqmck.jpg | 2315 images remaining
writting file: jiql1w.jpg | 2314 images remaining
writting file: jiqkli.png | 2313 images remaining
writting file: jiqhzh.jpg | 2312 images remaining
writting file: jiqeb6.jpg | 2311 images remaining
writting file: jiqbre.jpg | 2310 images remaining
writting file: jiq99z.jpg | 2309 images remaining
writting file: jiq8t3.jpg | 2308 images remaining
writting file: jiq8cf.jpg | 2307 images remaining
writting file: jiq78x.jpg | 2306 images remaining
writting file: jiq3jh.jpg | 2305 images remaining
writting file: jiq2ui.jpg | 2304 images remaining


writting file: jigbyt.jpg | 2159 images remaining
writting file: jig1fo.png | 2158 images remaining
writting file: jifxgh.jpg | 2157 images remaining
writting file: jifw9y.jpg | 2156 images remaining
writting file: jifvm9.png | 2155 images remaining
writting file: jifs91.jpg | 2154 images remaining
writting file: jifry2.jpg | 2153 images remaining
writting file: jifq6k.jpg | 2152 images remaining
writting file: jifokz.jpg | 2151 images remaining
writting file: jifoh5.png | 2150 images remaining
writting file: jifo05.jpg | 2149 images remaining
writting file: jifnza.jpg | 2148 images remaining
writting file: jifk5t.png | 2147 images remaining
writting file: jifi37.png | 2146 images remaining
writting file: jiffk3.png | 2145 images remaining
writting file: jife9d.jpg | 2144 images remaining
writting file: jifcif.jpg | 2143 images remaining
writting file: jifaw7.jpg | 2142 images remaining
writting file: jifaim.jpg | 2141 images remaining
writting file: jif9ty.jpg | 2140 images remaining


...

writting file: jg94ni.jpg | 840 images remaining
writting file: jg92wr.jpg | 839 images remaining
writting file: jg90w5.jpg | 838 images remaining
writting file: jg90to.jpg | 837 images remaining
writting file: jg90pz.jpg | 836 images remaining
writting file: jg8zme.jpg | 835 images remaining
writting file: jg8s3l.jpg | 834 images remaining
writting file: jg8nxm.png | 833 images remaining
writting file: jg8hp0.jpg | 832 images remaining
writting file: jg8f8v.jpg | 831 images remaining
writting file: jg80zc.jpg | 830 images remaining
writting file: jg7x96.jpg | 829 images remaining
writting file: jg7wfb.jpg | 828 images remaining
writting file: jg7vpd.jpg | 827 images remaining
writting file: jg7v3i.jpg | 826 images remaining
writting file: jg7u5g.jpg | 825 images remaining
writting file: jg7oc5.jpg | 824 images remaining
writting file: jg7njn.png | 823 images remaining
writting file: jg7n9e.png | 822 images remaining
writting file: jg7mdi.jpg | 821 images remaining
writting file: jg7kz

writting file: jfzhzv.jpg | 672 images remaining
writting file: jfzgi3.png | 671 images remaining
writting file: jfze23.jpg | 670 images remaining
writting file: jfzcpq.jpg | 669 images remaining
writting file: jfz60y.jpg | 668 images remaining
writting file: jfz5v7.png | 667 images remaining
writting file: jfz4te.jpg | 666 images remaining
writting file: jfyylr.jpg | 665 images remaining
writting file: jfyt9y.jpg | 664 images remaining
writting file: jfyqvs.jpg | 663 images remaining
writting file: jfypo5.png | 662 images remaining
writting file: jfykxd.png | 661 images remaining
writting file: jfyiyh.jpg | 660 images remaining
writting file: jfyi7z.png | 659 images remaining
writting file: jfyh6p.jpg | 658 images remaining
writting file: jfydl2.jpg | 657 images remaining
writting file: jfyc2z.jpg | 656 images remaining
writting file: jfybxz.jpg | 655 images remaining
writting file: jfyayp.jpg | 654 images remaining
writting file: jfya73.png | 653 images remaining
writting file: jfy4e

writting file: jfp6jx.png | 504 images remaining
writting file: jfp4wj.jpg | 503 images remaining
writting file: jfp1tv.jpg | 502 images remaining
writting file: jfp16q.jpg | 501 images remaining
writting file: jfozpg.jpg | 500 images remaining
writting file: jfoxh7.jpg | 499 images remaining
writting file: jfowub.jpg | 498 images remaining
writting file: jfovcs.jpg | 497 images remaining
writting file: jfoneb.jpg | 496 images remaining
writting file: jfokxt.jpg | 495 images remaining
writting file: jfoih9.png | 494 images remaining
writting file: jfofv2.png | 493 images remaining
writting file: jfocu9.jpg | 492 images remaining
writting file: jfoc0r.jpg | 491 images remaining
writting file: jfo6tk.jpg | 490 images remaining
writting file: jfo5cm.jpg | 489 images remaining
writting file: jfo0nc.jpg | 488 images remaining
writting file: jfo054.jpg | 487 images remaining
writting file: jfnxgl.png | 486 images remaining
writting file: jfnwne.jpg | 485 images remaining
writting file: jfnsw

writting file: jfgb0r.jpg | 335 images remaining
writting file: jfg8ab.jpg | 334 images remaining
writting file: jfg4io.png | 333 images remaining
writting file: jfg2wl.png | 332 images remaining
writting file: jffzd2.jpg | 331 images remaining
writting file: jffxst.jpg | 330 images remaining
writting file: jffvyq.jpg | 329 images remaining
writting file: jffsso.png | 328 images remaining
writting file: jffsf6.png | 327 images remaining
writting file: jffs3n.png | 326 images remaining
writting file: jffqlg.png | 325 images remaining
writting file: jffqhf.jpg | 324 images remaining
writting file: jffplm.png | 323 images remaining
writting file: jffnxd.jpg | 322 images remaining
writting file: jffljf.jpg | 321 images remaining
writting file: jffl7f.jpg | 320 images remaining
writting file: jffiwk.png | 319 images remaining
writting file: jffg5p.png | 318 images remaining
writting file: jffg4d.png | 317 images remaining
writting file: jffer5.jpg | 316 images remaining
writting file: jffe2

writting file: jf79nd.jpg | 167 images remaining
writting file: jf78ia.png | 166 images remaining
writting file: jf77ut.png | 165 images remaining
writting file: jf76pi.png | 164 images remaining
writting file: jf75uz.png | 163 images remaining
writting file: jf75om.jpg | 162 images remaining
writting file: jf718z.jpg | 161 images remaining
writting file: jf6qet.png | 160 images remaining
writting file: jf6nz6.jpg | 159 images remaining
writting file: jf6lpz.jpg | 158 images remaining
writting file: jf6h4d.jpg | 157 images remaining
writting file: jf6gmp.jpg | 156 images remaining
writting file: jf6gkk.jpg | 155 images remaining
writting file: jf667o.jpg | 154 images remaining
writting file: jf61vw.jpg | 153 images remaining
writting file: jf60f4.jpg | 152 images remaining
writting file: jf5ymu.jpg | 151 images remaining
writting file: jf5xhq.jpg | 150 images remaining
writting file: jf5vgm.jpg | 149 images remaining
writting file: jf5qf6.jpg | 148 images remaining
writting file: jf5q9

writting code to auto-delete all of the '[deleted]' images from my local drive. I have already updated the validation function to exclude these posts so I can use that to identify bad posts i've already downloaded too

In [88]:
bad_post_file_names = []
for post in submissions_raw:
    if not validate_submission(post):
        bad_post_file = f"{post.id}.{post.url[-3:]}"
        bad_post_file_names.append(bad_post_file)

In [90]:
len(bad_post_file_names) # yikes, thats a lot of placeholder images. could be a big part of why my model cant break 20% test accuracy

2106

In [91]:
bad_post_file_names[0]

'jvrgof.jpg'

In [97]:
for directory in ['digital', 'ink', 'pencil', 'paint', 'charcoal', 'mixed_medium', 'sculpture']:
    d_path = data_path / 'fsm_tts' / 'train' / directory
    with os.scandir(d_path) as scanner:
        for file in scanner:
            if file.name in bad_post_file_names:
                bad_path = d_path / file.name
                os.remove(bad_path)
                print(f'succesfully deleted {file.name} from {directory}')

succesfully deleted jhjpvx.jpg from digital
succesfully deleted jjrqyd.jpg from digital
succesfully deleted jk19cz.jpg from digital
succesfully deleted jgow5z.jpg from digital
succesfully deleted jkjc36.jpg from digital
succesfully deleted jpu3j2.jpg from digital
succesfully deleted jhp8bt.jpg from digital
succesfully deleted jpmx4q.jpg from digital
succesfully deleted jg4mfh.jpg from digital
succesfully deleted jm6gav.jpg from digital
succesfully deleted jkal78.jpg from digital
succesfully deleted jm0csa.jpg from digital
succesfully deleted jj178g.jpg from digital
succesfully deleted ju7l87.jpg from digital
succesfully deleted jv06t3.jpg from digital
succesfully deleted jkwwuh.jpg from digital
succesfully deleted jizxpf.jpg from digital
succesfully deleted js5g53.png from digital
succesfully deleted jucaya.jpg from digital
succesfully deleted jnb7rl.jpg from digital
succesfully deleted jq2f63.jpg from digital
succesfully deleted jniva6.png from digital
succesfully deleted jqj7fy.jpg f

succesfully deleted jlqfiq.jpg from digital
succesfully deleted jhubbi.png from digital
succesfully deleted jnasf0.jpg from digital
succesfully deleted jgawem.jpg from digital
succesfully deleted jryxbr.jpg from digital
succesfully deleted jicy5i.jpg from digital
succesfully deleted jv5e2y.png from digital
succesfully deleted jvd3gt.png from digital
succesfully deleted jiui18.jpg from digital
succesfully deleted jsbsv4.jpg from digital
succesfully deleted jpbrzv.jpg from digital
succesfully deleted jmmc0h.png from digital
succesfully deleted jl3dw1.jpg from digital
succesfully deleted jfe9lj.png from digital
succesfully deleted jsd08d.png from digital
succesfully deleted jpc4fr.jpg from digital
succesfully deleted jfokxt.jpg from digital
succesfully deleted jh8tfd.jpg from digital
succesfully deleted jf76pi.png from digital
succesfully deleted jt2cvf.jpg from digital
succesfully deleted jihtd8.jpg from digital
succesfully deleted jk95c2.png from digital
succesfully deleted jjw4c0.png f

succesfully deleted jgsjve.jpg from digital
succesfully deleted jn7fz8.jpg from digital
succesfully deleted jfmbeh.jpg from digital
succesfully deleted jrq1o0.jpg from digital
succesfully deleted jrf0nn.jpg from digital
succesfully deleted jpoqwa.jpg from digital
succesfully deleted jszn9m.png from digital
succesfully deleted jfxltp.jpg from digital
succesfully deleted jhum28.jpg from digital
succesfully deleted jjwf37.jpg from digital
succesfully deleted jm3yyb.jpg from digital
succesfully deleted jjcnp9.jpg from digital
succesfully deleted jhwgsn.jpg from digital
succesfully deleted jrrj15.jpg from digital
succesfully deleted jlv7j5.png from digital
succesfully deleted jid0p0.jpg from digital
succesfully deleted jo3n3t.jpg from digital
succesfully deleted jlvsdh.jpg from digital
succesfully deleted jgtblk.jpg from digital
succesfully deleted jpvvu0.jpg from digital
succesfully deleted jsqe2h.jpg from digital
succesfully deleted jq5n3s.jpg from digital
succesfully deleted jla54v.jpg f

succesfully deleted jt788q.jpg from ink
succesfully deleted jsmovw.jpg from ink
succesfully deleted jex8vw.jpg from ink
succesfully deleted jsj6mr.jpg from ink
succesfully deleted jfuup7.jpg from ink
succesfully deleted jhchyf.jpg from ink
succesfully deleted jqjig4.jpg from ink
succesfully deleted jj0bas.jpg from ink
succesfully deleted jtxp9v.jpg from ink
succesfully deleted jrdpl4.jpg from ink
succesfully deleted jke98q.jpg from ink
succesfully deleted jfrqf6.jpg from ink
succesfully deleted jtiwf4.jpg from ink
succesfully deleted jf8tuu.jpg from ink
succesfully deleted jnxash.jpg from ink
succesfully deleted jkyws6.jpg from ink
succesfully deleted juvb10.jpg from ink
succesfully deleted jstmpo.jpg from ink
succesfully deleted jtaqus.jpg from ink
succesfully deleted jsj9k8.jpg from ink
succesfully deleted jfwbke.jpg from ink
succesfully deleted jmo6d0.jpg from ink
succesfully deleted jmmuyo.jpg from ink
succesfully deleted jgcizt.jpg from ink
succesfully deleted jhekyb.jpg from ink


succesfully deleted jha388.jpg from paint
succesfully deleted jhxu0n.jpg from paint
succesfully deleted jsaa6t.jpg from paint
succesfully deleted jn5hxy.jpg from paint
succesfully deleted jhmeq4.jpg from paint
succesfully deleted jnwtb7.jpg from paint
succesfully deleted jku470.jpg from paint
succesfully deleted jkgj70.jpg from paint
succesfully deleted juallg.jpg from paint
succesfully deleted jt82km.jpg from paint
succesfully deleted jtm1t0.jpg from paint
succesfully deleted js2bw3.jpg from paint
succesfully deleted jm9rc2.jpg from paint
succesfully deleted jhyep0.jpg from paint
succesfully deleted jhkevf.jpg from paint
succesfully deleted jpfms6.jpg from paint
succesfully deleted jt2rcb.jpg from paint
succesfully deleted jm1or8.jpg from paint
succesfully deleted jsn1l2.jpg from paint
succesfully deleted jn080m.jpg from paint
succesfully deleted jum562.jpg from paint
succesfully deleted jscr4q.jpg from paint
succesfully deleted jmca3m.jpg from paint
succesfully deleted jvnmjk.jpg fro

succesfully deleted jkp8qy.jpg from mixed_medium
succesfully deleted jo0jix.png from mixed_medium
succesfully deleted jkgwjc.jpg from mixed_medium
succesfully deleted jnhyvc.jpg from mixed_medium
succesfully deleted jnza9s.jpg from mixed_medium
succesfully deleted junriw.jpg from mixed_medium
succesfully deleted jrxe9q.jpg from mixed_medium
succesfully deleted jrwg1l.jpg from sculpture
succesfully deleted jtiw5g.png from sculpture
succesfully deleted jn6nto.jpg from sculpture
succesfully deleted jsbx8o.jpg from sculpture
succesfully deleted jrwegn.jpg from sculpture
succesfully deleted jewxdr.jpg from sculpture
succesfully deleted jnheaw.jpg from sculpture
succesfully deleted jribw2.jpg from sculpture
succesfully deleted jkk6jm.jpg from sculpture
succesfully deleted jpu59y.jpg from sculpture
succesfully deleted jnzi3s.jpg from sculpture
succesfully deleted jtvpn3.jpg from sculpture
succesfully deleted jnhezv.jpg from sculpture
succesfully deleted jq3n9v.jpg from sculpture
succesfully d

In [190]:
# functions for locating posts by id for data exploration purposes

def find_submission_dict_by_id(id_str):
    for post in table_of_contents:
        if post['id'] == id_str:
            return post
        
def find_submission_obj_by_id(id_str):
    for post in table_of_contents:
        if post['id'] == id_str:
            return post

In [102]:
find_submission_dict_by_id('jiwi4i')

{'title': 'True Self, Reynold/Me, Sculpey Clay, 2020.',
 'medium': 'sculpture',
 'url': 'https://i.redd.it/oi6i4osc7lv51.jpg',
 'id': 'jiwi4i',
 'unix_time': 1603782535}

### class balancing

I'm combining charcoal and pencil into one category called 'non_ink_drawing' and adding 'chalk' as a keyword (this is a change to the get_wordlists() function above)

modifying original data acquisition functions to account for balancing

In [174]:
def fetch_balanced_submissions(n_to_fetch, date = int(dt.datetime.now().timestamp()), current_balance = None):
    class_counts = {}
    # taking into account the current class balance if data has already been downloaded
    if type(current_balance) == dict:
        class_counts = current_balance
    else:
        class_counts = dict(zip(list(get_wordlists().keys()), [0 for _ in get_wordlists().keys()]))
    start_epoch = date
    data_size = n_to_fetch + sum(list(class_counts.values()))
    n_per_class = int(data_size / len(class_counts.keys()))
    print((n_per_class, data_size))
    collected_posts = []
    while len(collected_posts) < n_to_fetch:
        print(f'polling pushshift for {n_to_fetch - len(collected_posts)} more posts before {start_epoch}')
        batch = list(s_api.search_submissions(before=start_epoch, subreddit='Art', limit=1000))
        for post in batch:
            if validate_submission(post):
                if class_counts[extract_medium_from_title(post.title)] <= n_per_class:
                    collected_posts.append(post)
                    class_counts[extract_medium_from_title(post.title)] += 1
        start_epoch = int(batch[-1].created_utc)
    return collected_posts

In [149]:
start_date = submissions_dict[-1]['unix_time']
current_b = {}
train_path = data_path / 'fsm_tts' / 'train'
with os.scandir(train_path) as t_scan:
    for class_name in t_scan:
        if class_name.is_dir():
            class_path = train_path / class_name.name
            current_b[class_name.name] = len(list(os.listdir(class_path)))
num_to_add = 3000

In [150]:
current_b

{'paint': 2906,
 'ink': 1987,
 'sculpture': 248,
 'non_ink_drawing': 540,
 'mixed_medium': 693,
 'digital': 3632}

In [130]:
new_submissions_raw = fetch_balanced_submissions(num_to_add, start_date, list(current_b.values()))

(1823, 10936)
polling pushshift for 3000 more posts before 1603221799
polling pushshift for 2981 more posts before 1603215291
polling pushshift for 2967 more posts before 1603207615
polling pushshift for 2959 more posts before 1603200103
polling pushshift for 2945 more posts before 1603119730
polling pushshift for 2930 more posts before 1603107022
polling pushshift for 2918 more posts before 1603091721
polling pushshift for 2909 more posts before 1603075687
polling pushshift for 2889 more posts before 1603063720
polling pushshift for 2872 more posts before 1603054332
polling pushshift for 2861 more posts before 1603047022
polling pushshift for 2842 more posts before 1603038535
polling pushshift for 2823 more posts before 1603031334
polling pushshift for 2804 more posts before 1603020340
polling pushshift for 2792 more posts before 1603005627
polling pushshift for 2780 more posts before 1602991019
polling pushshift for 2763 more posts before 1602979131
polling pushshift for 2750 more po

polling pushshift for 1086 more posts before 1601648156
polling pushshift for 1081 more posts before 1601639908
polling pushshift for 1078 more posts before 1601629455
polling pushshift for 1071 more posts before 1601614806
polling pushshift for 1062 more posts before 1601603241
polling pushshift for 1056 more posts before 1601594779
polling pushshift for 1047 more posts before 1601587263
polling pushshift for 1040 more posts before 1601579336
polling pushshift for 1033 more posts before 1601572803
polling pushshift for 1029 more posts before 1601567101
polling pushshift for 1025 more posts before 1601559992
polling pushshift for 1019 more posts before 1601552061
polling pushshift for 1016 more posts before 1601539141
polling pushshift for 1011 more posts before 1601523933
polling pushshift for 1006 more posts before 1601509966
polling pushshift for 1004 more posts before 1601500505
polling pushshift for 1001 more posts before 1601492272
polling pushshift for 993 more posts before 1601

polling pushshift for 377 more posts before 1600053813
polling pushshift for 369 more posts before 1600043877
polling pushshift for 364 more posts before 1600034720
polling pushshift for 356 more posts before 1600028384
polling pushshift for 351 more posts before 1600020951
polling pushshift for 345 more posts before 1600012047
polling pushshift for 344 more posts before 1600003611
polling pushshift for 341 more posts before 1599993256
polling pushshift for 338 more posts before 1599980307
polling pushshift for 336 more posts before 1599965199
polling pushshift for 327 more posts before 1599954990
polling pushshift for 323 more posts before 1599944947
polling pushshift for 316 more posts before 1599936483
polling pushshift for 313 more posts before 1599927905
polling pushshift for 310 more posts before 1599919877
polling pushshift for 306 more posts before 1599910388
polling pushshift for 302 more posts before 1599893995
polling pushshift for 298 more posts before 1599877895
polling pu

In [139]:
new_submissions_dict = [make_post_dict(post) for post in new_submissions_raw]

In [140]:
new_submissions_dict[0]

{'title': 'Even In Struggles Colors Burst Out To Make Life Colorful Once More,Hassan Patel,Ink,2020',
 'medium': 'ink',
 'url': 'https://i.redd.it/1zn773livau51.jpg',
 'id': 'jewkgr',
 'unix_time': 1603221634}

In [141]:
new_submissions_df = pd.DataFrame(new_submissions_dict)

In [142]:
new_submissions_df.medium.value_counts()

ink                1371
paint              1068
non_ink_drawing     383
sculpture           179
Name: medium, dtype: int64

downloading additional images identified with new balancing function

In [143]:
count = 0
for post in new_submissions_dict:
    response = requests.get(post['url'])
    file_name = f"{post['id']}.{post['url'][-3:]}"
    file_path = data_path / 'initial_dump' / post['medium'] / file_name
    if (count % 10) == 0:
        print(f'writting file: {file_name} | {len(new_submissions_dict) - count} images remaining')
    file = open(str(file_path), "wb")
    file.write(response.content)
    file.close()
    count += 1

writting file: jewkgr.jpg | 3001 images remaining
writting file: jevr5u.png | 2991 images remaining
writting file: jetiay.jpg | 2981 images remaining
writting file: jesg46.jpg | 2971 images remaining
writting file: jepf1d.jpg | 2961 images remaining
writting file: jendsk.jpg | 2951 images remaining
writting file: je1z0d.jpg | 2941 images remaining
writting file: jdzvbv.jpg | 2931 images remaining
writting file: jdx923.jpg | 2921 images remaining
writting file: jdu5qz.jpg | 2911 images remaining
writting file: jdsir7.jpg | 2901 images remaining
writting file: jdqrka.png | 2891 images remaining
writting file: jdozt9.jpg | 2881 images remaining
writting file: jdmtza.jpg | 2871 images remaining
writting file: jdllta.jpg | 2861 images remaining
writting file: jdk50l.jpg | 2851 images remaining
writting file: jdj2i8.jpg | 2841 images remaining
writting file: jdhvq0.jpg | 2831 images remaining
writting file: jdgu44.jpg | 2821 images remaining
writting file: jdfa7v.jpg | 2811 images remaining


writting file: j5vh5b.jpg | 1361 images remaining
writting file: j5tvw3.jpg | 1351 images remaining
writting file: j5s7ja.jpg | 1341 images remaining
writting file: j5q2bf.jpg | 1331 images remaining
writting file: j5o1bm.jpg | 1321 images remaining
writting file: j5mnf3.jpg | 1311 images remaining
writting file: j5ksxc.jpg | 1301 images remaining
writting file: j5iw1g.jpg | 1291 images remaining
writting file: j5h6oa.jpg | 1281 images remaining
writting file: j5ej0w.jpg | 1271 images remaining
writting file: j5bt8j.jpg | 1261 images remaining
writting file: j59qbh.jpg | 1251 images remaining
writting file: j58aj0.jpg | 1241 images remaining
writting file: j56vc3.jpg | 1231 images remaining
writting file: j558bj.jpg | 1221 images remaining
writting file: j53e0p.png | 1211 images remaining
writting file: j50qsg.jpg | 1201 images remaining
writting file: j4xsq5.jpg | 1191 images remaining
writting file: j4ueax.jpg | 1181 images remaining
writting file: j4qhhr.jpg | 1171 images remaining


In [164]:
all_submissions_dict = submissions_dict + new_submissions_dict + new_posts_3_dict

In [148]:
find_submission_dict_by_id('inaiyd')

{'title': 'Reflect, Me, Pen, 2020',
 'medium': 'ink',
 'url': 'https://i.redd.it/1dj2i4quoel51.jpg',
 'id': 'inaiyd',
 'unix_time': 1599345186}

moving extra samples in majority classes into a partitioned directory

In [152]:
train_path = data_path / 'fsm_tts' / 'train'
extras_path = data_path / 'leftover_samples'
with os.scandir(train_path) as t_scan:
    for class_name in t_scan:
        if class_name.is_dir():
            class_path = train_path / class_name.name
            new_d_path = extras_path / class_name.name
            while len(list(os.listdir(class_path))) > 1000:
                files = list(os.listdir(class_path))
                file = files[np.random.randint(low = 0, high = len(files))]
                file_path = class_path / file
                target_path = new_d_path / file
                shutil.move(file_path, target_path)
                print(f'successfuly moved {file_path} to {target_path} with {len(files) - 1000} more to go')

successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/paint/jqer9f.jpg to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/paint/jqer9f.jpg with 1906 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/paint/jacwgs.jpg to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/paint/jacwgs.jpg with 1905 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/paint/ji5lgv.jpg to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/paint/ji5lgv.jpg with 1904 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/paint/jpa2z7.jpg to /Users/matthewwilliamson/Documents/Flatiron/ca

successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/paint/juqg6z.jpg to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/paint/juqg6z.jpg with 1868 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/paint/jm9v8z.jpg to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/paint/jm9v8z.jpg with 1867 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/paint/jtyd83.png to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/paint/jtyd83.png with 1866 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/paint/jiptri.jpg to /Users/matthewwilliamson/Documents/Flatiron/ca

successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/paint/jkd9ht.jpg to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/paint/jkd9ht.jpg with 1814 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/paint/jpdwol.jpg to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/paint/jpdwol.jpg with 1813 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/paint/ja4bir.jpg to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/paint/ja4bir.jpg with 1812 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/paint/jqajsl.jpg to /Users/matthewwilliamson/Documents/Flatiron/ca

successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/paint/jmxdkx.jpg to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/paint/jmxdkx.jpg with 1755 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/paint/jjiwqe.jpg to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/paint/jjiwqe.jpg with 1754 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/paint/jhlqit.jpg to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/paint/jhlqit.jpg with 1753 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/paint/jt01su.jpg to /Users/matthewwilliamson/Documents/Flatiron/ca

successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/paint/jdjfem.jpg to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/paint/jdjfem.jpg with 1699 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/paint/jsu5u1.jpg to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/paint/jsu5u1.jpg with 1698 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/paint/j65xjt.jpg to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/paint/j65xjt.jpg with 1697 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/paint/jlnk5q.jpg to /Users/matthewwilliamson/Documents/Flatiron/ca

...

successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/digital/jmp0up.jpg to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/digital/jmp0up.jpg with 251 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/digital/jh5g7y.jpg to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/digital/jh5g7y.jpg with 250 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/digital/jtjke7.jpg to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/digital/jtjke7.jpg with 249 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/digital/jkronr.jpg to /Users/matthewwilliamson/Documents/

successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/digital/jmq0ke.png to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/digital/jmq0ke.png with 211 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/digital/jnazc1.jpg to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/digital/jnazc1.jpg with 210 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/digital/jt3h8t.jpg to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/digital/jt3h8t.jpg with 209 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/digital/jrwly7.jpg to /Users/matthewwilliamson/Documents/

successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/digital/jtyhp0.png to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/digital/jtyhp0.png with 164 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/digital/jvjljh.png to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/digital/jvjljh.png with 163 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/digital/jiyvx2.jpg to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/digital/jiyvx2.jpg with 162 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/digital/jpozjq.jpg to /Users/matthewwilliamson/Documents/

successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/digital/js9u1i.jpg to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/digital/js9u1i.jpg with 116 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/digital/jo2w0p.jpg to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/digital/jo2w0p.jpg with 115 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/digital/jsiry1.jpg to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/digital/jsiry1.jpg with 114 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/digital/jnfi92.jpg to /Users/matthewwilliamson/Documents/

successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/digital/jujpmo.png to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/digital/jujpmo.png with 65 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/digital/jsw82n.jpg to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/digital/jsw82n.jpg with 64 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/digital/jpljh7.jpg to /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/leftover_samples/digital/jpljh7.jpg with 63 more to go
successfuly moved /Users/matthewwilliamson/Documents/Flatiron/capstone_project/FlatironCapstone/data/fsm_tts/train/digital/jlixhs.jpg to /Users/matthewwilliamson/Documents/Fla

In [153]:
current_b = {}
with os.scandir(train_path) as t_scan:
    for class_name in t_scan:
        if class_name.is_dir():
            class_path = train_path / class_name.name
            current_b[class_name.name] = len(list(os.listdir(class_path)))
current_b

{'paint': 1000,
 'ink': 1000,
 'sculpture': 248,
 'non_ink_drawing': 540,
 'mixed_medium': 693,
 'digital': 1000}

In [154]:
num_new = 6000 - sum(list(current_b.values()))
print(num_new)
start_date = all_submissions_dict[-1]['unix_time']
print(start_date)

new_posts_3 = fetch_balanced_submissions(num_new, start_date, current_b.values())

1519
1599198884
(1001, 6000)
polling pushshift for 1519 more posts before 1599198884
polling pushshift for 1509 more posts before 1599188543
polling pushshift for 1506 more posts before 1599176448
polling pushshift for 1501 more posts before 1599168326
polling pushshift for 1497 more posts before 1599161077
polling pushshift for 1492 more posts before 1599153776
polling pushshift for 1485 more posts before 1599145956
polling pushshift for 1483 more posts before 1599138350
polling pushshift for 1478 more posts before 1599127971
polling pushshift for 1474 more posts before 1599112964
polling pushshift for 1467 more posts before 1599101559
polling pushshift for 1464 more posts before 1599091490
polling pushshift for 1455 more posts before 1599081449
polling pushshift for 1450 more posts before 1599074237
polling pushshift for 1443 more posts before 1599065460
polling pushshift for 1439 more posts before 1599057324
polling pushshift for 1432 more posts before 1599047304
polling pushshift f

polling pushshift for 848 more posts before 1597778156
polling pushshift for 841 more posts before 1597772162
polling pushshift for 835 more posts before 1597764136
polling pushshift for 830 more posts before 1597755118
polling pushshift for 827 more posts before 1597744137
polling pushshift for 821 more posts before 1597727991
polling pushshift for 813 more posts before 1597716439
polling pushshift for 803 more posts before 1597706864
polling pushshift for 802 more posts before 1597699504
polling pushshift for 801 more posts before 1597691731
polling pushshift for 796 more posts before 1597685020
polling pushshift for 791 more posts before 1597677091
polling pushshift for 789 more posts before 1597669994
polling pushshift for 786 more posts before 1597657603
polling pushshift for 783 more posts before 1597640550
polling pushshift for 778 more posts before 1597628181
polling pushshift for 774 more posts before 1597616242
polling pushshift for 772 more posts before 1597608617
polling pu

polling pushshift for 347 more posts before 1596393046
polling pushshift for 345 more posts before 1596386180
polling pushshift for 343 more posts before 1596380039
polling pushshift for 343 more posts before 1596371868
polling pushshift for 343 more posts before 1596366913
polling pushshift for 343 more posts before 1596354279
polling pushshift for 340 more posts before 1596346130
polling pushshift for 338 more posts before 1596339469
polling pushshift for 337 more posts before 1596332479
polling pushshift for 336 more posts before 1596324402
polling pushshift for 336 more posts before 1596316412
polling pushshift for 335 more posts before 1596310273
polling pushshift for 334 more posts before 1596302043
polling pushshift for 334 more posts before 1596297143
polling pushshift for 333 more posts before 1596293065
polling pushshift for 331 more posts before 1596283993
polling pushshift for 330 more posts before 1596276983
polling pushshift for 329 more posts before 1596269117
polling pu

polling pushshift for 149 more posts before 1595288221
polling pushshift for 148 more posts before 1595281880
polling pushshift for 146 more posts before 1595275329
polling pushshift for 146 more posts before 1595268779
polling pushshift for 146 more posts before 1595263418
polling pushshift for 145 more posts before 1595257168
polling pushshift for 144 more posts before 1595249192
polling pushshift for 144 more posts before 1595239890
polling pushshift for 142 more posts before 1595228485
polling pushshift for 141 more posts before 1595219764
polling pushshift for 141 more posts before 1595208620
polling pushshift for 140 more posts before 1595200725
polling pushshift for 137 more posts before 1595194398
polling pushshift for 137 more posts before 1595187847
polling pushshift for 137 more posts before 1595180768
polling pushshift for 136 more posts before 1595175191
polling pushshift for 133 more posts before 1595168188
polling pushshift for 133 more posts before 1595158677
polling pu

polling pushshift for 60 more posts before 1594044725
polling pushshift for 59 more posts before 1594036509
polling pushshift for 59 more posts before 1594026716
polling pushshift for 59 more posts before 1594015970
polling pushshift for 58 more posts before 1594004683
polling pushshift for 58 more posts before 1593996453
polling pushshift for 56 more posts before 1593989163
polling pushshift for 55 more posts before 1593982112
polling pushshift for 55 more posts before 1593974946
polling pushshift for 55 more posts before 1593969164
polling pushshift for 55 more posts before 1593963324
polling pushshift for 54 more posts before 1593954397
polling pushshift for 53 more posts before 1593940906
polling pushshift for 53 more posts before 1593930903
polling pushshift for 53 more posts before 1593917343
polling pushshift for 53 more posts before 1593907232
polling pushshift for 53 more posts before 1593899447
polling pushshift for 52 more posts before 1593891746
polling pushshift for 51 mor

polling pushshift for 11 more posts before 1592695874
polling pushshift for 11 more posts before 1592686826
polling pushshift for 11 more posts before 1592679095
polling pushshift for 10 more posts before 1592672776
polling pushshift for 10 more posts before 1592666043
polling pushshift for 9 more posts before 1592656637
polling pushshift for 9 more posts before 1592645277
polling pushshift for 9 more posts before 1592635089
polling pushshift for 9 more posts before 1592623215
polling pushshift for 9 more posts before 1592614134
polling pushshift for 7 more posts before 1592606260
polling pushshift for 7 more posts before 1592599054
polling pushshift for 7 more posts before 1592591857
polling pushshift for 7 more posts before 1592584973
polling pushshift for 7 more posts before 1592577061
polling pushshift for 7 more posts before 1592566551
polling pushshift for 7 more posts before 1592553870
polling pushshift for 6 more posts before 1592540779
polling pushshift for 6 more posts before

In [157]:
new_posts_3_dict = [make_post_dict(post) for post in new_posts_3]
new_posts_3_df = pd.DataFrame(new_posts_3_dict)
new_posts_3_df

,title,medium,url,id,unix_time
0,"Morning Training Exercise, me, acrylic, 2020",paint,https://i.redd.it/aiun2z1pj2l51.jpg,imadnp,1599198169
1,"Temple of Nobody, Me, 3d, 2020",digital,https://i.redd.it/slgkt7i2j2l51.jpg,imac2i,1599197963
2,"""The Unboxing of Pandora"", by Hellevan (me), A...",paint,https://i.redd.it/k2x4yxdyi2l51.jpg,imabso,1599197921
3,"Lady by Gold, me, digital, 2020",digital,https://i.redd.it/hhskl6zki2l51.jpg,imaar2,1599197794
4,"""Extinct"", me, ink on paper, 2020",ink,https://i.redd.it/r6fpmh2ob2l51.jpg,im9skm,1599195470
...,...,...,...,...,...
1514,"Velociraptor, me, clay, 2019-2020",sculpture,https://i.redd.it/pomgkfwdgq551.jpg,hbmxat,1592514054
1515,"Meg, Me, Vitreous Paint and enamel on glass, 2020",sculpture,https://i.redd.it/i2285uovtp551.jpg,hbkjf9,1592506490
1516,"Pocket Assurbanipal's library, me, presssoft c...",sculpture,https://i.redd.it/yi0d5f1pmp551.jpg,hbjy3f,1592504094
1517,"“After David”, Jim Parks (my dad), wire, 2020",sculpture,https://i.redd.it/merof5d5oo551.jpg,hbg8d5,1592492450


In [159]:
new_posts_3_df.medium.value_counts()

ink                754
non_ink_drawing    462
sculpture          297
mixed_medium         2
paint                2
digital              2
Name: medium, dtype: int64

In [162]:
count = 0
for post in new_posts_3_dict:
    response = requests.get(post['url'])
    file_name = f"{post['id']}.{post['url'][-3:]}"
    file_path = data_path / 'initial_dump' / post['medium'] / file_name
    if (count % 10) == 0:
        print(f'writting file: {file_name} | {len(new_posts_3_dict) - count} images remaining')
    file = open(str(file_path), "wb")
    file.write(response.content)
    file.close()
    count += 1

writting file: imadnp.jpg | 1519 images remaining
writting file: im7t6j.jpg | 1509 images remaining
writting file: im0twd.jpg | 1499 images remaining
writting file: ilx1ax.jpg | 1489 images remaining
writting file: ilr1fa.jpg | 1479 images remaining
writting file: illyr9.jpg | 1469 images remaining
writting file: ilgb3n.jpg | 1459 images remaining
writting file: ilcxzd.jpg | 1449 images remaining
writting file: il7q9j.jpg | 1439 images remaining
writting file: il2zed.jpg | 1429 images remaining
writting file: iksp2u.jpg | 1419 images remaining
writting file: iknln4.jpg | 1409 images remaining
writting file: ikl9k1.jpg | 1399 images remaining
writting file: ikc360.jpg | 1389 images remaining
writting file: ik5qeb.jpg | 1379 images remaining
writting file: ik1an0.jpg | 1369 images remaining
writting file: ijxi8g.jpg | 1359 images remaining
writting file: ijql0d.jpg | 1349 images remaining
writting file: ijlsuo.jpg | 1339 images remaining
writting file: ijghq5.jpg | 1329 images remaining


In [169]:
current_b = {}
with os.scandir(train_path) as t_scan:
    for class_name in t_scan:
        if class_name.is_dir():
            class_path = train_path / class_name.name
            current_b[class_name.name] = len(list(os.listdir(class_path)))
current_b

{'paint': 1000,
 'ink': 1000,
 'sculpture': 545,
 'non_ink_drawing': 1000,
 'mixed_medium': 695,
 'digital': 1000}

In [166]:
len(all_submissions_dict)

14556

In [171]:
num_new = 6000 - sum(list(current_b.values()))
print(num_new)
start_date = all_submissions_dict[-1]['unix_time']
print(start_date)

new_posts_4 = fetch_balanced_submissions(num_new, start_date, current_b)

760
1592414197
(1000, 6000)
polling pushshift for 760 more posts before 1592414197
polling pushshift for 750 more posts before 1592408681
polling pushshift for 748 more posts before 1592401240
polling pushshift for 743 more posts before 1592393836
polling pushshift for 740 more posts before 1592381318
polling pushshift for 737 more posts before 1592369059
polling pushshift for 731 more posts before 1592358626
polling pushshift for 728 more posts before 1592349515
polling pushshift for 721 more posts before 1592342431
polling pushshift for 718 more posts before 1592335850
polling pushshift for 712 more posts before 1592329606
polling pushshift for 704 more posts before 1592322728
polling pushshift for 695 more posts before 1592316241
polling pushshift for 693 more posts before 1592308807
polling pushshift for 688 more posts before 1592297500
polling pushshift for 687 more posts before 1592286522
polling pushshift for 683 more posts before 1592276826
polling pushshift for 679 more posts 

polling pushshift for 395 more posts before 1591206401
polling pushshift for 395 more posts before 1591201133
polling pushshift for 395 more posts before 1591192897
polling pushshift for 394 more posts before 1591184072
polling pushshift for 393 more posts before 1591173516
polling pushshift for 393 more posts before 1591165998
polling pushshift for 393 more posts before 1591155865
polling pushshift for 392 more posts before 1591147355
polling pushshift for 392 more posts before 1591138692
polling pushshift for 392 more posts before 1591130346
polling pushshift for 391 more posts before 1591124044
polling pushshift for 391 more posts before 1591117836
polling pushshift for 390 more posts before 1591111020
polling pushshift for 390 more posts before 1591104620
polling pushshift for 390 more posts before 1591095961
polling pushshift for 389 more posts before 1591086833
polling pushshift for 389 more posts before 1591075388
polling pushshift for 389 more posts before 1591066622
polling pu

polling pushshift for 351 more posts before 1589967915
polling pushshift for 351 more posts before 1589958155
polling pushshift for 351 more posts before 1589947957
polling pushshift for 351 more posts before 1589939350
polling pushshift for 351 more posts before 1589932435
polling pushshift for 350 more posts before 1589925216
polling pushshift for 350 more posts before 1589919283
polling pushshift for 350 more posts before 1589913477
polling pushshift for 348 more posts before 1589906439
polling pushshift for 348 more posts before 1589900957
polling pushshift for 347 more posts before 1589894490
polling pushshift for 346 more posts before 1589885218
polling pushshift for 345 more posts before 1589875823
polling pushshift for 344 more posts before 1589866008
polling pushshift for 344 more posts before 1589857741
polling pushshift for 344 more posts before 1589851599
polling pushshift for 344 more posts before 1589846844
polling pushshift for 344 more posts before 1589839278
polling pu

polling pushshift for 298 more posts before 1588780490
polling pushshift for 297 more posts before 1588773792
polling pushshift for 297 more posts before 1588763336
polling pushshift for 297 more posts before 1588752479
polling pushshift for 297 more posts before 1588741231
polling pushshift for 297 more posts before 1588732639
polling pushshift for 297 more posts before 1588724759
polling pushshift for 295 more posts before 1588717930
polling pushshift for 295 more posts before 1588712519
polling pushshift for 295 more posts before 1588706518
polling pushshift for 295 more posts before 1588701289
polling pushshift for 295 more posts before 1588694893
polling pushshift for 295 more posts before 1588688562
polling pushshift for 295 more posts before 1588681836
polling pushshift for 295 more posts before 1588673888
polling pushshift for 294 more posts before 1588662083
polling pushshift for 293 more posts before 1588651348
polling pushshift for 292 more posts before 1588643340
polling pu

polling pushshift for 253 more posts before 1587603534
polling pushshift for 253 more posts before 1587596092
polling pushshift for 252 more posts before 1587590785
polling pushshift for 252 more posts before 1587586339
polling pushshift for 252 more posts before 1587581863
polling pushshift for 252 more posts before 1587576087
polling pushshift for 252 more posts before 1587571420
polling pushshift for 251 more posts before 1587565823
polling pushshift for 251 more posts before 1587558549
polling pushshift for 251 more posts before 1587550410
polling pushshift for 251 more posts before 1587540952
polling pushshift for 251 more posts before 1587533654
polling pushshift for 251 more posts before 1587525896
polling pushshift for 251 more posts before 1587516986
polling pushshift for 251 more posts before 1587509778
polling pushshift for 250 more posts before 1587502962
polling pushshift for 250 more posts before 1587497259
polling pushshift for 250 more posts before 1587490754
polling pu

polling pushshift for 201 more posts before 1586520298
polling pushshift for 201 more posts before 1586512658
polling pushshift for 199 more posts before 1586502523
polling pushshift for 199 more posts before 1586490799
polling pushshift for 199 more posts before 1586482826
polling pushshift for 199 more posts before 1586475608
polling pushshift for 198 more posts before 1586468652
polling pushshift for 197 more posts before 1586462208
polling pushshift for 197 more posts before 1586456332
polling pushshift for 197 more posts before 1586449793
polling pushshift for 197 more posts before 1586443462
polling pushshift for 196 more posts before 1586434315
polling pushshift for 195 more posts before 1586423517
polling pushshift for 195 more posts before 1586411109
polling pushshift for 195 more posts before 1586403988
polling pushshift for 195 more posts before 1586397871
polling pushshift for 194 more posts before 1586390030
polling pushshift for 192 more posts before 1586383419
polling pu

polling pushshift for 142 more posts before 1585367986
polling pushshift for 142 more posts before 1585360622
polling pushshift for 142 more posts before 1585352603
polling pushshift for 142 more posts before 1585347252
polling pushshift for 141 more posts before 1585342089
polling pushshift for 141 more posts before 1585334417
polling pushshift for 141 more posts before 1585329010
polling pushshift for 141 more posts before 1585321887
polling pushshift for 141 more posts before 1585313880
polling pushshift for 141 more posts before 1585306337
polling pushshift for 141 more posts before 1585292437
polling pushshift for 141 more posts before 1585280185
polling pushshift for 141 more posts before 1585271447
polling pushshift for 141 more posts before 1585264705
polling pushshift for 140 more posts before 1585258711
polling pushshift for 140 more posts before 1585252728
polling pushshift for 140 more posts before 1585246800
polling pushshift for 140 more posts before 1585241941
polling pu

polling pushshift for 101 more posts before 1584121426
polling pushshift for 101 more posts before 1584112659
polling pushshift for 101 more posts before 1584103356
polling pushshift for 101 more posts before 1584090624
polling pushshift for 101 more posts before 1584076261
polling pushshift for 101 more posts before 1584060734
polling pushshift for 101 more posts before 1584051600
polling pushshift for 101 more posts before 1584044254
polling pushshift for 101 more posts before 1584037201
polling pushshift for 99 more posts before 1584029723
polling pushshift for 99 more posts before 1584021668
polling pushshift for 98 more posts before 1584011526
polling pushshift for 98 more posts before 1583995240
polling pushshift for 98 more posts before 1583980759
polling pushshift for 96 more posts before 1583968861
polling pushshift for 95 more posts before 1583962264
polling pushshift for 95 more posts before 1583955331
polling pushshift for 95 more posts before 1583949033
polling pushshift f

polling pushshift for 36 more posts before 1582642189
polling pushshift for 36 more posts before 1582635543
polling pushshift for 35 more posts before 1582621175
polling pushshift for 34 more posts before 1582606216
polling pushshift for 34 more posts before 1582596723
polling pushshift for 33 more posts before 1582587169
polling pushshift for 32 more posts before 1582579255
polling pushshift for 32 more posts before 1582572966
polling pushshift for 32 more posts before 1582567104
polling pushshift for 31 more posts before 1582560083
polling pushshift for 31 more posts before 1582549962
polling pushshift for 30 more posts before 1582537128
polling pushshift for 30 more posts before 1582525205
polling pushshift for 28 more posts before 1582514905
polling pushshift for 28 more posts before 1582505833
polling pushshift for 28 more posts before 1582499175
polling pushshift for 27 more posts before 1582492724
polling pushshift for 27 more posts before 1582486154
polling pushshift for 27 mor

In [172]:
new_posts_4_dict = [make_post_dict(post) for post in new_posts_4]
new_posts_4_df = pd.DataFrame(new_posts_4_dict)
new_posts_4_df

,title,medium,url,id,unix_time
0,"Art or shoes, ME, digital, 2020",digital,https://i.redd.it/65ls3kib7i551.jpg,havkg1,1592414153
1,"NGC6188, me, acrylic, 2017",paint,https://i.redd.it/7a3buaso6i551.jpg,havhsp,1592413938
2,"Couple by the Pool, Kelly Beeman, ink & waterc...",mixed_medium,https://i.redd.it/9u9643au5i551.jpg,havfnp,1592413760
3,"Human, me, pencil, 2020",mixed_medium,https://i.redd.it/eigntkz06i551.jpg,havff0,1592413741
4,"Dance, Me, Oil on Metal, 2020",mixed_medium,https://i.redd.it/pf8k44ec4i551.jpg,havb1n,1592413395
...,...,...,...,...,...
756,"Traffic Jam, by Jaclyn Kraemer, metal and leat...",sculpture,https://i.redd.it/9tm92dkl7jh41.jpg,f5dmyd,1581965997
757,"Iridescent Scorpion, Me, Stained Glass and Sol...",sculpture,https://i.redd.it/16fp4mputih41.jpg,f5cen7,1581961381
758,"Iridescent Scorpion, Me, Stained Glass, 2020- ...",sculpture,https://i.redd.it/lm7nnumntih41.jpg,f5cdze,1581961315
759,"Tree of reflection, Me, wire art, 2019",sculpture,https://i.redd.it/t3j9gsexqgh41.jpg,f57095,1581936180


In [175]:
new_posts_4_df.medium.value_counts()

sculpture          451
mixed_medium       306
ink                  1
paint                1
digital              1
non_ink_drawing      1
Name: medium, dtype: int64

In [176]:
all_submissions_dict = all_submissions_dict + new_posts_4_dict
len(all_submissions_dict)

15317

In [177]:
count = 0
for post in new_posts_4_dict:
    response = requests.get(post['url'])
    file_name = f"{post['id']}.{post['url'][-3:]}"
    file_path = data_path / 'initial_dump' / post['medium'] / file_name
    if (count % 10) == 0:
        print(f'writting file: {file_name} | {len(new_posts_4_dict) - count} images remaining')
    file = open(str(file_path), "wb")
    file.write(response.content)
    file.close()
    count += 1

writting file: havkg1.jpg | 761 images remaining
writting file: hatoq4.jpg | 751 images remaining
writting file: hamzsr.jpg | 741 images remaining
writting file: hagj5k.jpg | 731 images remaining
writting file: hac6d3.png | 721 images remaining
writting file: ha8djd.jpg | 711 images remaining
writting file: ha5wwo.jpg | 701 images remaining
writting file: ha1oqo.jpg | 691 images remaining
writting file: h9tf13.jpg | 681 images remaining
writting file: h9oerh.jpg | 671 images remaining
writting file: h9lz6b.jpg | 661 images remaining
writting file: h9i1lw.jpg | 651 images remaining
writting file: h99o1l.jpg | 641 images remaining
writting file: h92du5.jpg | 631 images remaining
writting file: h8ymb3.jpg | 621 images remaining
writting file: h8usm1.jpg | 611 images remaining
writting file: h8o1i0.jpg | 601 images remaining
writting file: h8icm6.jpg | 591 images remaining
writting file: h8d72w.jpg | 581 images remaining
writting file: h8bh3t.jpg | 571 images remaining
writting file: h87vc

In [182]:
current_b = {}
with os.scandir(train_path) as t_scan:
    for class_name in t_scan:
        if class_name.is_dir():
            class_path = train_path / class_name.name
            current_b[class_name.name] = len(list(os.listdir(class_path)))
current_b

{'paint': 951,
 'ink': 951,
 'sculpture': 951,
 'non_ink_drawing': 951,
 'mixed_medium': 951,
 'digital': 951}

In [184]:
find_submission_dict_by_id('jj36zf')

{'title': 'Mermaid Octopus, Me, Pencil, 2020',
 'medium': 'mixed_medium',
 'url': 'https://i.redd.it/kbwazv9ynnv51.jpg',
 'id': 'jj36zf',
 'unix_time': 1603812327}

this is a big oopsies. looks like theres a bug in the medium extraction function that classifies anything with the word pencil as 'mixed_medium' guess i have to fix that and then re-categorize my dataset

having fixed the bug in the medium extractor, I'll search through my complete list of posts that I've downloaded and create a new table_of_contents list of dictionaries

In [186]:
all_submissions_raw = submissions_raw + new_submissions_raw + new_posts_3 + new_posts_4

In [187]:
len(all_submissions_raw)

15317

making sure to exclude the meta-data for the posts that have broken images since those images have already been removed from my downloaded data

In [209]:
table_of_contents = [make_post_dict(post) for post in all_submissions_raw if validate_submission(post)]

In [210]:
find_submission_dict_by_id('jj36zf')

{'title': 'Mermaid Octopus, Me, Pencil, 2020',
 'medium': 'non_ink_drawing',
 'url': 'https://i.redd.it/kbwazv9ynnv51.jpg',
 'id': 'jj36zf',
 'unix_time': 1603812327,
 'post': Submission(id='jj36zf')}

In [211]:
toc_df = pd.DataFrame(table_of_contents)

In [212]:
toc_df.medium.value_counts()

digital            4049
paint              3040
ink                1945
non_ink_drawing    1760
sculpture          1009
mixed_medium        561
Name: medium, dtype: int64

now that I have all the meta-data properly sorted, I see that mixed medium is a pretty significant minority class. I'm choosing to drop these data points instead of pulling more from reddit since my model is already having a hard enough time making valid predictions. I won't delete the images or their meta-data though, just sequester them into a folder of unused photos.

the next step will be to move all of my images into a single 'dump' folder and then write some code to sort them into train/val/test folders. since 'sculpture' is my smallest class at 1009 images, i'll make a train set with 800 images per class, a test set with 100 images per class, and a validation set with 109 images per class.

In [ ]:
train_path = data_path / 'tts' / 'train'
test_path = data_path / 'tts' / 'test'
val_path = data_path / 'tts' / 'val'
extras_path = data_path / 'leftover_samples'
with os.scandir(train_path) as t_scan:
    for class_name in t_scan:
        if class_name.is_dir():
            class_path = train_path / class_name.name
            new_d_path = extras_path / class_name.name
            while len(list(os.listdir(class_path))) > 1000:
                files = list(os.listdir(class_path))
                file = files[np.random.randint(low = 0, high = len(files))]
                file_path = class_path / file
                target_path = new_d_path / file
                shutil.move(file_path, target_path)
                print(f'successfuly moved {file_path} to {target_path} with {len(files) - 1000} more to go')